In [0]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [0]:
# import required libraries
from ast import literal_eval
from utils.data import DataPipeline, generate_data

In [0]:
# Getting parameters for data code
data_params = {
    'file_path': dbutils.widgets.get('file_path'),
    'holdout': dbutils.widgets.get('holdout'),
    'test_size': float(dbutils.widgets.get('test_size')),
    'holdout_size': float(dbutils.widgets.get('holdout_size')),
    'stratify': literal_eval(dbutils.widgets.get('stratify'))
}

In [0]:
data_obj = generate_data(**data_params)
train, test, holdout = data_obj.generate_train_data(), data_obj.generate_test_data(), data_obj.generate_holdout_data()

In [0]:
train.head()

In [0]:
test.head()

In [0]:
holdout.head()